In [60]:
import numpy as np
import copy
import astropy
import astropy.units as u
import astropy.constants as c
import astropy.cosmology
import photontools

In [54]:
class Transient(object):
    def __init__(self):
        self.name = None
        self.instrument = None
        self.Nphoton = None
        self.redshift = None
        self.luminosity_distance = None
        self.Eb_v = None
        self.maxdate = None
        self.bands = None
        self.data = None # flux/mag with the same order as self.bands [N_band][N_time, 3] 3: time, flux/mag, error of flux/mag
        self.spectra = None # just for stock json data
        
def calc_model_lc_for_one_transient(spectra, transient, filter, convert_to_magnitude=False):
    spectra_ = copy.deepcopy(spectra)
    spectra_ = spectra_.redshift(z=transient.redshift)
    spectra_ = spectra_.dust_extinction(Eb_v = transient.Eb_v, model="maeda")
    lc = photontools.calc_band_flux(spectra_, filter)
    if (convert_to_magnitude):
        if (u.get_physical_type((transient.luminosity_distance * u.m / u.m).unit) == "length"):
            lc = lc.convert_flux_to_magnitude(filter, system="AB", distance=transient.luminosity_distance)
        elif (u.get_physical_type((transient.luminosity_distance * u.m / u.m).unit) == "dimensionless"):
            lc = lc.convert_flux_to_magnitude(filter, system="AB", distance=transient.luminosity_distance * u.Mpc)
        else:
            raise ValueError("Input luminosity_distnace unit is wrong!")
    else:
        lc.data = lc.data / (4 * np.pi * transient.luminosity_distance.cgs.value**2)
    return lc


In [47]:
class Reader(object):
    def read_decam_one_transient(name, df_header, directory_light_curves=None, cosmo=None):
        """
        return Transient with the given name and df_header

        arguments
        =========
        cosmo: type: astropy.cosmology. If None, we will use it given astropy.cosmology.Planck15. 
                     used to calc luminosity distance
        """

        def assign_luminosity_distance(redshift, cosmo):
            return cosmo.luminosity_distance(redshift)

        if directory_light_curves is None:
            directory_light_curves = "/Users/kawana/GoogleDrive/sync/study/yoshidalab/tidal_disruption/imbh-wd/nuclear/std/compare/snaps/successed/photons/0002_WD_TDE_radiative_transfer/catalog/fast_transients/Pursiainen_2018_DES/LCs/"
        if cosmo is None:
            cosmo = astropy.cosmology.Planck15

        df_this = df_header[df_header["name"] == name].iloc[0]
        transient = Transient()
        transient.name = name
        transient.redshift = df_this["zspec"]
        transient.luminosity_distance = assign_luminosity_distance(transient.redshift, cosmo)
        transient.Eb_v = df_this["Eb_v"]
        transient.bands = np.array(["g", "r", "i", "z"])

        transient.data = [[]] * transient.bands.size

        for i, band in enumerate(transient.bands):
            fpath = directory_light_curves + "/" + name + "_" + band + ".dat"
            data_ = np.loadtxt(fpath)
            data2_ = data_.T
            data2_[1:] = data2_[1:] * 1e-18
            transient.data[i] = data2_
            transient.data

        return transient

In [53]:
class Converter(object):
    def replace_obs_frame_to_rest_frame(transient:Transient, band="g"):
        new_transient = copy.deepcopy(transient)

        mask_ = transient.bands == band
        index_of_the_band = np.arange(mask_.size)[mask_][0]
        index_of_band_peak = np.argmax(transient.data[index_of_the_band][1])
        date_peak = transient.data[index_of_the_band][0][index_of_band_peak]

        for i in range(len(transient.data)):
            new_transient.data[i][0] = (transient.data[i][0] - date_peak) / (1 + transient.redshift)
            new_transient.data[i][1] = transient.data[i][1] * (1 + transient.redshift)
            new_transient.data[i][2] = transient.data[i][2] * (1 + transient.redshift)

        return new_transient

    def replace_MJD_to_obs_frame_day_after_peak(transient:Transient, band="g"):
        new_transient = copy.deepcopy(transient)

        mask_ = transient.bands == band
        index_of_the_band = np.arange(mask_.size)[mask_][0]
        index_of_band_peak = np.argmax(transient.data[index_of_the_band][1])
        date_peak = transient.data[index_of_the_band][0][index_of_band_peak]

        for i in range(len(transient.data)):
            new_transient.data[i][0] = (transient.data[i][0] - date_peak)

        return new_transient


In [ ]:
class Plotter(object):

    def plot_model(fig, ax, spectra, transient, time_shift = 0., **kwargs):
    
    lc = calc_model_lc(spectra, transient, filter_decam)[9:]
    
    for i, theta in enumerate(lc.thetas):
        for j, phi in enumerate(lc.phis):
            for k, band in enumerate(lc.bands):
                if band != "y" and band != "Y":
                    ax.plot((lc.times - time_shift) * lc.Doppler_shift_intrinsic[i,j], lc.data[:,i,j,k], color=dict_color[band], **kwargs)
    return fig, ax


#     def plot_decam_transient(fig, ax, transient, **kwargs):

#         for i, band in enumerate(transient.bands):
#             # detection
#             ax.errorbar(transient.data[i][0], transient.data[i][1], transient.data[i][2], color=dict_color[band], label=band, fmt="o", **kwargs)
#     # #         upper limit
#     #         df_ = transient.data[i][transient.data[i]["mag_upper"]]
#     #         ax.errorbar(df_["Phase"], df_["mag"], fmt="v", color=dict_color[band], label="")        
#         return fig, ax

    def plot_decam_transient(fig, ax, transient, **kwargs):
        upper_limit_sigma = 3.
        for i, band in enumerate(transient.bands):
            flag_detection = transient.data[i][1] - upper_limit_sigma * transient.data[i][2] > 0
            # detection
    #         ax.errorbar(transient.data[i][0][flag_detection], transient.data[i][1][flag_detection], transient.data[i][2][flag_detection], color=dict_color[band], label=band, fmt=dict_marker[band], capsize=4, **kwargs)
            ax.errorbar(transient.data[i][0][flag_detection], transient.data[i][1][flag_detection], transient.data[i][2][flag_detection], color=dict_color[band], label=band, fmt=dict_marker[band], capsize=4, **kwargs)

    #         upper limit
            ax.errorbar(transient.data[i][0][~flag_detection], transient.data[i][1][~flag_detection], transient.data[i][2][~flag_detection], color=dict_color[band], label="", fmt="v", fillstyle="none", capsize=4, **kwargs)
    #         ax.errorbar(transient.data[i][0][~flag_detection], transient.data[i][1][~flag_detection] + upper_limit_sigma * transient.data[i][2][~flag_detection], color=dict_color[band], label="", fmt="v", fillstyle="none", capsize=4, **kwargs)
        return fig, ax

def plot_decam_transient_and_model(fig, ax, transient, spectra, time_shift, savefig=False, **keys):

    fig, ax = plot_decam_transient(fig, ax, transient)

    xlim_ = ax.get_xlim()
    ylim_ = ax.get_ylim()
    
    plot_model(fig, ax, spectra, transient, time_shift=time_shift, **keys)

    plt.xlim(xlim_)
    # plt.ylim(ylim_)

    plt.yscale("log")
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    plt.grid()
    plt.legend()
    plt.title(transient.name + " (z = {:.2f})".format(transient.redshift))
    plt.xlabel(r"Rest frame days since g$_{\rm max}$")
    plt.ylabel(r"Flux [$10^{-18}$ erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$]")
    if (transient.name != "DES14S2anq"):
        plt.ylim(1e-20, 1e-17)
    else:
        plt.ylim(1e-19, 1e-16)
    if (savefig):
        fig.tight_layout()
        plt.savefig("figs/obs/rapid/Pursiainen_2018/{}_paper2.pdf".format(transient.name), dpi=300, transparent=True)
    plt.show()
    plt.close()
    return


In [ ]:

# def chi_square(transient, lc, time_shift, amplify):
#     """
#     return chi_square[lc.thetas.size, lc.phis.size], reduced_chi_square[lc.thetas.size, lc.phis.size]
#     """
#     N_params = 2

#     index_band = np.zeros(transient.bands.size, dtype=int)
#     for i, band in enumerate(transient.bands):
#         for j, lc_band in enumerate(lc.bands):
#             if (band == lc_band):
#                 index_band[i] = j

#     dof = -N_params * np.ones(lc.data.shape[1:3], dtype=int)
#     chi_square = np.zeros(lc.data.shape[1:3], dtype=float)

#     for i, band in enumerate(transient.bands):
#         model_lc = lc[:,:,:,index_band[i]].interpolate_time(transient.data[i][0] - time_shift, left=np.nan, right=np.nan, consider_Doppler_shift=True)
        
#         # if model light curve is not given at some time, we use sigma=infty there
#         mask_nan = np.isnan(model_lc.data[:,:,:,0])
#         dof += np.sum(~mask_nan, axis=0)
#         transient_sigma = np.tile(transient.data[i][2], (lc.thetas.size, lc.phis.size, 1))
#         transient_sigma[mask_nan.swapaxes(0,2).swapaxes(0,1)] = np.infty
#         model_lc.data[mask_nan] = 0.

#         chi_square[:,:] += np.sum(((transient.data[i][1] - amplify * model_lc.data[:,:,:,0].swapaxes(0, 2).swapaxes(0, 1)) / transient_sigma)**2, axis=2)

#     return chi_square, chi_square / dof
# #     return chi_square, chi_square / dof, model_lc


# def find_best_fit(transient, lc, param_ranges, way_optimize="shgo", **kwargs):
#     """
#     Now, assume that parameters are [time_shift, amplify]

#     arguments
#     =========    
#     param_ranges: [[param1_min, param1_max], [param2_min, param2_max], ...]
    

# """
    
    
#     ways_avail_optimize = ["shgo"]
#     if not(np.sum([way_optimize == way_avail for way_avail in ways_avail_optimize])):
#         raise ValueError("way_optimize {} is not supported now".format(way_optimize))
    
#     if (way_optimize == "shgo"):
#         results = scipy.optimize.shgo(lambda x: chi_square(transient, lc, x[0], x[1])[1].min(),
#                                       bounds=param_ranges, **kwargs)
#         popt = results["x"]
        
#     chi2, chi2_reduced = chi_square(transient, lc, *popt)
#     return popt, chi2, chi2_reduced


def chi_square(transient, lc, time_shift):
    """
    return chi_square[lc.thetas.size, lc.phis.size], reduced_chi_square[lc.thetas.size, lc.phis.size]
    """
    N_params = 1

    index_band = np.zeros(transient.bands.size, dtype=int)
    for i, band in enumerate(transient.bands):
        for j, lc_band in enumerate(lc.bands):
            if (band == lc_band):
                index_band[i] = j

    dof = -N_params * np.ones(lc.data.shape[1:3], dtype=int)
    chi_square = np.zeros(lc.data.shape[1:3], dtype=float)

    for i, band in enumerate(transient.bands):
        model_lc = lc[:,:,:,index_band[i]].interpolate_time(transient.data[i][0] - time_shift, left=np.nan, right=np.nan, consider_Doppler_shift=True)
        
        # if model light curve is not given at some time, we use sigma=infty there
        mask_nan = np.isnan(model_lc.data[:,:,:,0])
        dof += np.sum(~mask_nan, axis=0)
        transient_sigma = np.tile(transient.data[i][2], (lc.thetas.size, lc.phis.size, 1))
        transient_sigma[mask_nan.swapaxes(0,2).swapaxes(0,1)] = np.infty
        model_lc.data[mask_nan] = 0.

        chi_square[:,:] += np.sum(((transient.data[i][1] - model_lc.data[:,:,:,0].swapaxes(0, 2).swapaxes(0, 1)) / transient_sigma)**2, axis=2)

    return chi_square, chi_square / dof
#     return chi_square, chi_square / dof, model_lc


def find_best_fit(transient, lc, param_ranges, way_optimize="shgo", **kwargs):
    """
    Now, assume that parameters are [time_shift, amplify]

    arguments
    =========    
    param_ranges: [[param1_min, param1_max], [param2_min, param2_max], ...]
    

"""
    
    
    ways_avail_optimize = ["shgo"]
    if not(np.sum([way_optimize == way_avail for way_avail in ways_avail_optimize])):
        raise ValueError("way_optimize {} is not supported now".format(way_optimize))
    
    if (way_optimize == "shgo"):
        results = scipy.optimize.shgo(lambda x: chi_square(transient, lc, x[0])[1].min(),
                                      bounds=param_ranges, **kwargs)
        popt = results["x"]
        
    chi2, chi2_reduced = chi_square(transient, lc, *popt)
    return popt, chi2, chi2_reduced



In [ ]:
# def plot_model_with_fit(fig, ax, spectra, transient, *args, **kwargs):
#     """
#     Now assume that 
#     args = [time_shift, amplify]
#     """
#     time_shift, amplify = args[0], args[1]
    
#     # redshift and dust_extinction
#     lc = calc_model_lc(spectra, transient, filter_decam)[9:]
    
#     for i, theta in enumerate(lc.thetas):
#         for j, phi in enumerate(lc.phis):
#             for k, band in enumerate(lc.bands):
#                 if band != "y" and band != "Y":
#                     ax.plot(time_shift + lc.times * lc.Doppler_shift_intrinsic[i,j], amplify * lc.data[:,i,j,k], color=dict_color[band], **kwargs)
#     return fig, ax

# def plot_one_transient_obs_and_model_with_fit(fig, ax, transient, spectra, chi2_reduced, *args, savefig=False, enable_amplify=False, plot_best_angle=True, **kwargs):
#     """
#     Now assume that 
#     args = [time_shift, amplify]
#     """
#     time_shift, amplify = args[0], args[1]

#     fig, ax = plot_decam_transient(fig, ax, transient, alpha=1)

#     xlim_ = ax.get_xlim()
#     ylim_ = ax.get_ylim()
    
#     plot_model_with_fit(fig, ax, spectra, transient, *args, **kwargs)

#     plt.xlim(xlim_)
#     # plt.ylim(ylim_)

#     plt.yscale("log")
#     ax.xaxis.set_minor_locator(AutoMinorLocator())
#     plt.grid()
#     plt.legend()
#     title_str = transient.name + " (z = {:.2f})\n".format(transient.redshift)
#     title_str += r"$t_{\rm shift} =$" + r"${:.1f}$ d, Amplify$ = {:.1f}$, $\chi^2_r = {:.1f}$".format(time_shift, amplify, chi2_reduced.min())
    
#     plt.xlabel(r"Rest frame days since g$_{\rm max}$")
#     plt.ylabel(r"Flux [erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$]")
#     plt.title(title_str)
    
#     if(plot_best_angle):
#         index_best_angle = np.unravel_index(np.argmin(chi2_reduced), chi2_reduced.shape)
#         plot_model_with_fit(fig, ax, spectra_hewd[:, index_best_angle[0], index_best_angle[1],:], transient_, *popt, alpha=1, linewidth=2)
            
#     if (transient.name != "DES14S2anq"):
#         plt.ylim(1e-20, 1e-17)
#     else:
#         plt.ylim(1e-19, 1e-16)
#     if (savefig):
#         fig.tight_layout()
# #         plt.savefig("figs/obs/rapid/Pursiainen_2018/t_A/{}.png".format(transient.name), dpi=300, transparent=True)
#         plt.savefig("figs/obs/rapid/Pursiainen_2018/vary_time/{}.png".format(transient.name), dpi=300, transparent=True)
#     plt.show()
#     plt.close()
#     return

def plot_model_with_fit(fig, ax, spectra, transient, *args, **kwargs):
    """
    Now assume that 
    args = [time_shift]
    """
    time_shift = args[0]
    
    # redshift and dust_extinction
    lc = calc_model_lc(spectra, transient, filter_decam)[9:]
    
    for i, theta in enumerate(lc.thetas):
        for j, phi in enumerate(lc.phis):
            for k, band in enumerate(lc.bands):
                if band != "y" and band != "Y":
                    ax.plot(time_shift + lc.times * lc.Doppler_shift_intrinsic[i,j], lc.data[:,i,j,k], color=dict_color[band], **kwargs)
    return fig, ax

def plot_one_transient_obs_and_model_with_fit(fig, ax, transient, spectra, chi2_reduced, *args, savefig=False, enable_amplify=False, plot_best_angle=True, **kwargs):
    """
    Now assume that 
    args = [time_shift, amplify]
    """
    time_shift = args[0]

    fig, ax = plot_decam_transient(fig, ax, transient, alpha=1, zorder=10)

    xlim_ = ax.get_xlim()
    ylim_ = ax.get_ylim()
    
    plot_model_with_fit(fig, ax, spectra, transient, *args, label="", **kwargs)

    plt.xlim(xlim_)
    # plt.ylim(ylim_)

    plt.yscale("log")
    ax.xaxis.set_minor_locator(AutoMinorLocator())
#     plt.grid()
    plt.legend().set_zorder(20)
    title_str = transient.name + " (z = {:.2f})".format(transient.redshift)
#     title_str += r"\n$t_{\rm shift} =$" + r"${:.1f}$ d, $\chi^2_r = {:.1f}$".format(time_shift, chi2_reduced.min())
    
#     plt.xlabel(r"Rest frame days since g$_{\rm max}$")
    plt.xlabel(r"Observed days since g$_{\rm max}$")
    plt.ylabel(r"Observed Flux [erg cm$^{-2}$ s$^{-1}$ ${\rm \AA}^{-1}$]")
    plt.title(title_str)
    
    if(plot_best_angle):
        index_best_angle = np.unravel_index(np.argmin(chi2_reduced), chi2_reduced.shape)
        plot_model_with_fit(fig, ax, spectra_hewd[:, index_best_angle[0], index_best_angle[1],:], transient_, *popt, alpha=0.8, linewidth=2)
            
    if (transient.name != "DES14S2anq"):
        plt.ylim(1e-20, 1e-17)
    else:
        plt.ylim(1e-19, 1e-16)
    if (savefig):
        fig.tight_layout()
#         plt.savefig("figs/obs/rapid/Pursiainen_2018/t_A/{}.png".format(transient.name), dpi=300, transparent=True)
        plt.savefig("figs/obs/rapid/Pursiainen_2018/vary_time/{}_paper_vel_44Ti_corrected.pdf".format(transient.name), dpi=300, transparent=True)
    plt.show()
    plt.close()
    return


In [ ]:
def plot_panstarrs(fig, ax, name, df_header, df_lc):
    
    def get_information_from_df_header(name, df_header):
        df_this = df_header[df_header["name"] == name].iloc[0]
        transient = Transient()
        transient.name = name
        transient.redshift = df_this["z"]
        transient.luminosity_distance = df_this["distance"] # Mpc
        transient.Eb_v = df_this["Eb_v"]
        transient.bands = np.array(["g", "r", "i", "z"])
        transient.data = [[]] * transient.bands.size
        
        return transient

    def get_light_curve_from_df_lc(name, transient, df_lc):
#         transient.bands
        df_this = df_lc[df_lc["name"] == name].sort_values("MJD")
        for i, band in enumerate(transient.bands):
            df_band = df_this[[df_this.loc[j, "Filter"][0] == band for j in df_this.index]]
            transient.data[i] = df_band[["MJD", "Phase", "mag_upper", "mag", "emag"]]
            
        return transient
    
    transient = get_information_from_df_header(name, df_header)
    transient = get_light_curve_from_df_lc(name, transient, df_lc)
    
    for i, band in enumerate(transient.bands):
        # detection
        df_ = transient.data[i][~transient.data[i]["mag_upper"]]
        ax.errorbar(df_["Phase"], df_["mag"], df_["emag"], fmt="--o", color=dict_color[band], label=band)
#         upper limit
        df_ = transient.data[i][transient.data[i]["mag_upper"]]
        ax.errorbar(df_["Phase"], df_["mag"], fmt="v", color=dict_color[band], label="")
        
    return transient, fig, ax

def plot_model(fig, ax, lc, time_shift = 0., **keys):
    for i, theta in enumerate(lc.thetas):
        for j, phi in enumerate(lc.phis):
            for k, band in enumerate(lc.bands):
                if band != "y":
                    ax.plot((lc.times - time_shift) * lc.Doppler_shift_intrinsic[i,j], lc.data[:,i,j,k], color=dict_color[band], **keys)
    return fig, ax

In [ ]:
def read_one_json(fpath):
    with open(fpath) as f:
        transient = Transient()
        json_dict = json.load(f)
        transient.name = list(json_dict.keys())[0]
        transient.Eb_v = np.float(json_dict[transient.name]["ebv"][0]["value"])
        transient.maxdate = astropy.time.Time(json_dict[transient.name]["maxdate"][0]["value"].replace("/", "-")).mjd

        if (json_dict[transient.name]["lumdist"][0]["u_value"] == "Mpc"):
            transient.luminosity_distance = np.float(json_dict[transient.name]["lumdist"][0]["value"]) * u.Mpc
        else:
            raise ValueError ("lumdist units {} is not supported yet!".format(json_dict[transient.name]["lumdist"]["u_value"]))
        transient.redshift = np.float(json_dict[transient.name]["redshift"][0]["value"])

        # for check E_B-V
        display(json_dict[transient.name]["ebv"])

        transient.spectra = json_dict[transient.name]["spectra"]

        lc = pd.DataFrame(json_dict[transient.name]["photometry"])
        lc["band"].replace(np.nan, "blank", inplace=True)
        transient.bands = np.sort([np.unicode(b) for b in np.unique(lc["band"])])
        transient.data = [[]] * transient.bands.size
        for i, band in enumerate(transient.bands):
            transient.data[i] = lc[lc["band"] == band]
            transient.data[i] = transient.data[i].sort_values("time")
            transient.data[i] = transient.data[i].reset_index(drop=True)
        transient.Nphoton = len(json_dict[transient.name]["photometry"])
        return transient

def read_all_json(dpath):
    fpaths = subprocess.getoutput('find {} -name "*.json" | sort').split("\n")
    transients = [[]] * len(fpaths)
    for i, fpath in enumerate(fpaths):
        transients[i] = read_one_json(fpath)
    
    return transients




In [ ]:
def plot_model2(fig, ax, lc_model, transient, time_shift = 0., use_mosfit_color=False, **keys):
    
    def find_bands_duplicated(lc_model, transient):
        """Todo: modify not only [0] but e.g. Swift band"""
        bands_all = pd.Series(np.append([band[0] for band in lc_model.bands], 
                                        [band.replace("'", "") for band in transient.bands]))
        return bands_all[bands_all.duplicated()].to_numpy()
    
    bands_duplicated = find_bands_duplicated(lc_model, transient)
    print(bands_duplicated)
    
    for i, theta in enumerate(lc_model.thetas):
        for j, phi in enumerate(lc_model.phis):
            for k, band in enumerate(lc_model.bands):
                band = band[0]
                if band in bands_duplicated:
                    if not (use_mosfit_color):
                        ax.plot(transient.maxdate - time_shift + (lc_model.times) * lc_model.Doppler_shift_intrinsic[i,j], lc_model.data[:,i,j,k], color=dict_color[band], **keys)
                    else:
                        ax.plot(transient.maxdate - time_shift + (lc_model.times) * lc_model.Doppler_shift_intrinsic[i,j], lc_model.data[:,i,j,k], color=bandcolorf(band), **keys)
    return fig, ax


In [ ]:
def plot_one_astrocats(fig, ax, fpath_json, use_mosfit_color=False, dict_color=None):

#     sns.reset_orig()
    with open(fpath, 'r', encoding = 'utf-8') as f:
        data = json.loads(f.read())
        if 'name' not in data:
            data = data[list(data.keys())[0]]

    photo = data['photometry']

    real_data = len([x for x in photo if 'band' in x and 'magnitude' in x and (
        'realization' not in x or 'simulated' in x)]) > 0

    band_attr = ['band', 'instrument', 'telescope', 'system', 'bandset']
    band_list = list(set([tuple(x.get(y, '')
                                for y in band_attr) for x in photo
                                if 'band' in x and 'magnitude' in x]))
    real_band_list = list(set([tuple(x.get(y, '')
                                     for y in band_attr) for x in photo
                                     if 'band' in x and 'magnitude' in x and (
                                         'realization' not in x or 'simulated' in x)]))
    xray_instrument_attr = ['instrument', 'telescope']
    xray_instrument_list = list(set([tuple(x.get(y, '')
                                for y in xray_instrument_attr) for x in photo
                                if 'instrument' in x and 'countrate' in x])) 
    real_band_list = list(set([tuple(x.get(y, '')
                                     for y in band_attr) for x in photo
                                     if 'band' in x and 'magnitude' in x and (
                                         'realization' not in x or 'simulated' in x)]))
    real_xray_instrument_list = list(set([tuple(x.get(y, '')
                                     for y in xray_instrument_attr) for x in photo
                                     if 'instrument' in x and 'countrate' in x and (
                                         'realization' not in x or 'simulated' in x)]))

    # Uncomment line below to only plot from the specified instruments.
    # inst_exclusive_list = ['UVOT']

#     fig = plt.figure(figsize=(12,8))
#     ax.invert_yaxis()
    # ax.set_xlim(55700,55820)
    # ax.set_ylim(bottom=25, top=19)
    ax.set_xlabel('MJD')
    ax.set_ylabel('Apparent Magnitude')
    used_bands = []
    for full_band in tqdm_notebook(band_list, desc='Photo', leave=False):
        (band, inst, tele, syst, bset) = full_band
        try:
            inst_exclusive_list
        except:
            pass
        else:
            if inst not in inst_exclusive_list:
                continue
        extra_nice = ', '.join(list(filter(None, OrderedDict.fromkeys((inst, syst, bset)).keys())))
#         nice_name = band + ((' [' + extra_nice + ']') if extra_nice else '')
        nice_name = band

    #     realizations = [[] for x in range(len(model['realizations']))]
        for ph in photo:
            rn = ph.get('realization', None)
            si = ph.get('simulated', False)
            if rn and not si:
                if tuple(ph.get(y, '') for y in band_attr) == full_band:
                    realizations[int(rn) - 1].append((
                        float(ph['time']), float(ph['magnitude']), [
                            float(ph.get('e_lower_magnitude', ph.get('e_magnitude', 0.0))),
                            float(ph.get('e_upper_magnitude', ph.get('e_magnitude', 0.0)))],
                    ph.get('upperlimit')))
    #     numrz = np.sum([1 for x in realizations if len(x)])
    #     for rz in realizations:
    #         if not len(rz):
    #             continue
    #         xs, ys, vs, us = zip(*rz)
    #         label = '' if full_band in used_bands or full_band in real_band_list else nice_name
    #         if max(vs) == 0.0:
    #             plt.plot(xs, ys, color=bandcolorf(band),
    #                              label=label, linewidth=0.5)
    #         else:
    #             xs = np.array(xs)
    #             ymi = np.array(ys) - np.array([np.inf if u else v[0] for v, u in zip(vs, us)])
    #             yma = np.array(ys) + np.array([v[1] for v in vs])
    #             plt.fill_between(xs, ymi, yma, color=bandcolorf(band), edgecolor=None,
    #                              label=label, alpha=1.0/numrz, linewidth=0.0)
    #             plt.plot(xs, ys, color=bandcolorf(band), 
    #                              label=label, alpha=1.0, linewidth=0.5)
    #         if label:
    #             used_bands = list(set(used_bands + [full_band]))
        if real_data:
            for s in range(2):
                if s == 0:
                    cond = False
                    symb = 'o'
                else:
                    cond = True
                    symb = 'v'
                vec = [(float(x['time']), float(x['magnitude']),
                        0.0 if 'upperlimit' in x else float(x.get('e_lower_magnitude', x.get('e_magnitude', 0.0))),
                        float(x.get('e_upper_magnitude', x.get('e_magnitude', 0.0)))) for x in photo
                       if 'magnitude' in x and ('realization' not in x or 'simulated' in x) and
                       'host' not in x and 'includeshost' not in x and
                       x.get('upperlimit', False) == cond and
                       tuple(x.get(y, '') for y in band_attr) == full_band]
                if not len(vec):
                    continue
                xs, ys, yls, yus = zip(*vec)
#                 label = nice_name if full_band not in used_bands else ''
                label = nice_name.replace("'", "") if band not in used_bands else ''
                if(use_mosfit_color):
                    color = bandcolorf(band)
                else:
                    color = dict_color[band.replace("'", "")]
                plt.errorbar(xs, ys, yerr=(yus, yls), color=color, fmt=symb,
                             label=label,
                             markeredgecolor='black', markeredgewidth=1, capsize=1,
                             elinewidth=1.5, capthick=2, zorder=10)
                plt.errorbar(xs, ys, yerr=(yus, yls), color='k', fmt=symb, capsize=2,
                             elinewidth=2.5, capthick=3, zorder=5)
                if label:
                    used_bands = list(set(used_bands + [band]))
#                     used_bands = list(set(used_bands + [full_band]))
#     plt.margins(0.02, 0.1)
#     plt.grid()
#     plt.title(fpath.split("/")[-1][:-5])
#     plt.show()
#     plt.close()
    # fig.savefig('../products/lc.pdf')
    
    

In [ ]:
def binning_spectra_in_wavelength(spectra:photontools.Spectra, wavelength_bin_sep=1):
    new_spectra = copy.deepcopy(spectra)
    wavelengths_with_edge = np.append(np.arange(wavelength_bin_sep * np.int(np.min(spectra.wavelengths) / wavelength_bin_sep),
                                        wavelength_bin_sep * np.int(np.max(spectra.wavelengths) / wavelength_bin_sep),
                                        wavelength_bin_sep
                                       ), wavelength_bin_sep * np.int(np.max(spectra.wavelengths) / wavelength_bin_sep))
    wavelengths_center = 0.5 * (wavelengths_with_edge[1:] + wavelengths_with_edge[:-1])
    weight = np.zeros((wavelengths_center.size, spectra.wavelengths.size-1), dtype=float)
    new_data = np.zeros(wavelengths_center.size, dtype=float)
    for i in range(wavelengths_center.size):
        for j in range(spectra.wavelengths.size-1):
            weight[i, j] = np.max([0, np.min([wavelengths_with_edge[i+1], spectra.wavelengths[j+1]])
                                 - np.max([wavelengths_with_edge[i], spectra.wavelengths[j]])])
    new_data = np.sum(spectra.data[:,:,:,:-1] * weight, axis=(-1))
    new_data = np.sum(spectra.data[:,:,:,:-1] * weight, axis=(-1))
    new_spectra.wavelengths = wavelengths_center
    new_spectra.data = new_data / wavelength_bin_sep
    return new_spectra
#     new_spectra.wavelengths = np.exp(np.log(spectra.wavelengths.reshape(N_wavelength_bins, every_wavelength)).mean(axis=1))
#     last_wavelength  = np.exp(np.log(spectra.wavelengths[-1]) + np.diff(np.log(spectra.wavelengths))[-2])
#     wavelength_delta = np.diff(np.append(spectra.wavelengths, last_wavelength))

#     data = data.reshape(N_time_bins,
#                         N_theta_bins, 
#                         N_phi_bins, 
#                         N_wavelength_bins,
#                         every_wavelength
#                        )
#     # weighted mean of data for wavelength & time
#     data = np.sum(data * wavelength_delta.reshape(N_wavelength_bins, every_wavelength), axis=-1) / wavelength_delta.reshape(N_wavelength_bins, every_wavelength).sum(axis=1)
